In [42]:
import numpy as np
from collections import defaultdict

In [43]:
class Component:
    def __init__(self, name : str, node1 : int, node2 : int, current_value : float, initial_value : float):
        self.name = name
        self.node1 = node1
        self.node2 = node2
        self.current_value = current_value
        self.initial_value = initial_value
    
    def __str__(self):
        return f"{self.name} is connected between node{self.node1} and node{self.node2}. Current value is {self.current_value}. Initial value is {self.initial_value}"

In [110]:
def read_file(filePath):
    file = open(filePath, 'r')
    lines = file.readlines()
    
    sim_time = float(lines[0])
    num_iter = float(lines[1])
    nodes = set()
    
    circuit_components = defaultdict(list)
    
    for line in lines[2:-1]:
        name, node1, node2, current_value, initial_value = line.strip('\n').split(' ')
        nodes.add(node1)
        nodes.add(node2)
        
        component = Component(name, int(node1[-1]), int(node2[-1]), float(current_value), float(initial_value))
        circuit_components[name].append(component)
        
    return sim_time, num_iter, circuit_components, len(nodes) - 1, len(circuit_components['Vsrc'])

In [135]:
def construct_Gmat(n, resistances):
    G = np.zeros((n, n))
    for res in resistances:
        G[res.node1 - 1][res.node1 - 1] += (1/res.current_value)
        if res.node2 != 0:
            G[res.node2 - 1][res.node2 - 1] += (1/res.current_value)
            G[res.node1 - 1][res.node2 - 1] -= (1/res.current_value)
            G[res.node2 - 1][res.node1 - 1] -= (1/res.current_value)
    return G

In [136]:
# Read all needed data
simulation_time, num_iter, circuit_components, n, m =  read_file('./testcases/1.txt')

print(n, m)

3 1


In [138]:
# Construct G matrix
construct_Gmat(n, circuit_components['R'])

array([[ 0.5  ,  0.   ,  0.   ],
       [ 0.   ,  0.375, -0.25 ],
       [ 0.   , -0.25 ,  0.25 ]])